# XGBoost算法案例实训 - 信用评分模型

## 案例背景  

为了降低不良贷款率，保障自身资金安全，提高风险控制水平，银行等金融机构会根据客户的信用历史资料构建信用评分模型给客户评分。根据客户的信用得分，可以估计客户按时还款的可能，并据此决定是否发放贷款及贷款的额度和利率。

## 一、多元线性回归模型

### 1.读取数据

In [1]:
import pandas as pd
df = pd.read_excel('/home/mw/input/XG3004/信用评分卡模型.xlsx')
df.head()

,月收入,年龄,性别,历史授信额度,历史违约次数,信用评分
0,7783,29,0,32274,3,73
1,7836,40,1,6681,4,72
2,6398,25,0,26038,2,74
3,6483,23,1,24584,4,65
4,5167,23,1,6710,3,73


### 2.提取特征变量和目标变量

In [2]:
# 通过如下代码将特征变量和目标变量单独提取出来
X = df.drop(columns='信用评分')
Y = df['信用评分']

### 3.模型训练及搭建

In [3]:
# 从Scikit-Learn库中引入LinearRegression()模型进行模型训练
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X,Y)

LinearRegression()

In [4]:
print('各系数为:' + str(model.coef_))
print('常数项系数k0为:' + str(model.intercept_))

各系数为:[ 5.58658996e-04  1.62842002e-01  2.18430276e-01  6.69996665e-05
 -1.51063940e+00]
常数项系数k0为:67.16686603853383


### 4.模型评估

In [5]:
# 利用模型评估的方法对此多元线性回归模型进行评估
import statsmodels.api as sm
X2 = sm.add_constant(X)
est = sm.OLS(Y, X2).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   信用评分   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     337.6
Date:                Sat, 30 Sep 2023   Prob (F-statistic):          2.32e-211
Time:                        19:04:14   Log-Likelihood:                -2969.8
No. Observations:                1000   AIC:                             5952.
Df Residuals:                     994   BIC:                             5981.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         67.1669      1.121     59.906      0.000      64.967      69.367
月收入            0.0006   8.29e-05      6.735      0.000       0.000       0.001
年龄             0.1628      0.022      7.420      0.000       0.120       0.206
性别             0.2184      0.299      0.730      0.466      -0.369       0.806
历史授信额度        6.7e-05   7.78e-06      8.609      0.000    5.17e-05    8.23e-05
历史违约次数        -1.5106      0.140    -10.811      0.000      -1.785      -1.236
==============================================================================
Omnibus:                       13.180   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               12.534
Skew:                          -0.236   Prob(JB):                      0.00190
Kurtosis:                       2.721   Cond. No.                     4.27e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.27e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

可以看到模型整体的R-squared为0.629，Adj. R-Squared为0.628，整体拟合效果一般，可能是因为数据量偏少的原因。同时我们再来观察P值，可以发现大部分特征变量的P值都较小（小于0.05），的确是和目标变量：信用评分显著相关，而性别这一特征变量的P值达到了0.466，即与目标变量没有显著相关性，这个也的确符合经验认知，所以在多元线性回归模型中，我们其实可以把性别这一特征变量舍去。

## 二、GBDT回归模型

In [6]:
# 这里使用GBDT回归模型做回归分析，首先读取1000条信用卡客户的数据并划分特征变量和目标变量，这部分代码和上面线性回归的代码是一样的。

import pandas as pd
df = pd.read_excel('/home/mw/input/XG3004/信用评分卡模型.xlsx')

X = df.drop(columns='信用评分')
y = df['信用评分']

### 1.划分训练集和测试集

In [7]:
# 划分训练集和测试集数据
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

### 2.模型训练及搭建

In [8]:
# 从Scikit-Learn库中引入GBDT模型进行模型训练
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()  # 使用默认参数
model.fit(X_train, y_train)

GradientBoostingRegressor()

### 3.模型预测及评估

In [9]:
# 模型搭建完毕后，通过如下代码预测测试集数据
y_pred = model.predict(X_test)
print(y_pred[0:10])

[70.77631652 71.40032104 73.73465155 84.52533945 71.09188294 84.9327599
 73.72232388 83.44560704 82.61221486 84.86927209]


In [10]:
# 将预测值和实际值进行对比：
a = pd.DataFrame()  # 创建一个空DataFrame 
a['预测值'] = list(y_pred)
a['实际值'] = list(y_test)
a.head()

,预测值,实际值
0,70.776317,79
1,71.400321,80
2,73.734652,62
3,84.525339,89
4,71.091883,80


In [11]:
# 因为GradientBoostingRegressor()是一个回归模型，所以我们通过查看其R-squared值来评判模型的拟合效果：
from sklearn.metrics import r2_score
r2 = r2_score(y_test, model.predict(X_test))
print(r2)

0.6774854432215086


第1行代码从Scikit-Learn库中引入r2_score()函数；第2行代码将训练集的真实值和模型预测值传入r2_score()函数，得出R-squared评分为0.675，可以看到这个结果较线性回归模型获得的0.629是有所改善的。

In [12]:
# 我们还可以通过GradientBoostingRegressor()自带的score()函数来查看模型预测的效果：
model.score(X_test, y_test)

0.6774854432215086

## 三、XGBoost回归模型

In [13]:
# 1.读取数据
import pandas as pd
df = pd.read_excel('/home/mw/input/XG3004/信用评分卡模型.xlsx')
# 2.提取特征变量和目标变量
X = df.drop(columns='信用评分')
y = df['信用评分']
# 3.划分测试集和训练集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [14]:
# 4.模型训练及搭建
from xgboost import XGBRegressor
model = XGBRegressor()  # 使用默认参数
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [15]:
# 5.模型预测及评估
y_pred = model.predict(X_test)
print(y_pred[0:10])

[74.62306  69.01495  76.393486 83.88998  71.5683   86.257324 76.0784
 81.38994  81.05504  83.24717 ]


In [16]:
# 将预测值和实际值进行对比：
a = pd.DataFrame()  # 创建一个空DataFrame 
a['预测值'] = list(y_pred)
a['实际值'] = list(y_test)
a.head()

,预测值,实际值
0,74.623062,79
1,69.014954,80
2,76.393486,62
3,83.889977,89
4,71.568298,80


In [17]:
# 因为XGBRegressor()是一个回归模型，所以通过查看R-squared来评判模型的拟合效果：
from sklearn.metrics import r2_score
r2 = r2_score(y_test, model.predict(X_test))
print(r2)

0.5715437436791975


In [18]:
# 我们还可以通过XGBRegressor()自带的score()函数来查看模型预测的效果：
model.score(X_test, y_test)

0.5715437436791975

In [19]:
# 6.查看特征重要性
features = X.columns  # 获取特征名称
importances = model.feature_importances_  # 获取特征重要性

# 通过二维表格形式显示
importances_df = pd.DataFrame()
importances_df['特征名称'] = features
importances_df['特征重要性'] = importances
importances_df.sort_values('特征重要性', ascending=False)

,特征名称,特征重要性
0,月收入,0.324461
4,历史违约次数,0.307467
3,历史授信额度,0.202864
1,年龄,0.098869
2,性别,0.066339


**补充知识点1：XGBoost回归模型的参数调优**

In [20]:
# 对XGBoost回归模型进行参数调优，代码如下：
from sklearn.model_selection import GridSearchCV  
parameters = {'max_depth': [1, 3, 5], 'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.05, 0.1, 0.2]}  # 指定模型中参数的范围
clf = XGBRegressor()  # 构建回归模型
grid_search = GridSearchCV(model, parameters, scoring='r2', cv=5) 

这里唯一需要注意的是最后一行代码中的scoring参数需要设置成'r2'，其表示的是R-squared值，因为是回归模型，所以参数调优时应该选择R-squared值来进行评判，而不是分类模型中常用的准确度'accuracy'或者ROC曲线对应的AUC值'roc_auc'。  
通过如下代码获取最优参数：

In [21]:
grid_search.fit(X_train, y_train)  # 传入数据
grid_search.best_params_  # 输出参数的最优值

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}

获得最优参数如下所示：  
{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 50}

In [22]:
# 在模型中设置参数，代码如下：
model = XGBRegressor(max_depth=3, n_estimators=50, learning_rate=0.1)
model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=3, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=50, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [23]:
# 此时再通过r2_score()函数进行模型评估，代码如下（也可以用model.score(X_test, y_test)进行评分，效果一样）：
from sklearn.metrics import r2_score
r2 = r2_score(y_test, model.predict(X_test))
print(r2)

0.6884486054771359


此时获得的R-squared值如下所示：  
0.688。  
可以看到调参后的R-squared值优于未调参前的R-squared值0.678。

### 对于XGBoost模型，有必要做很多数据预处理吗？

在传统的机器模型中，我们往往需要做挺多的数据预处理，例如数据的归一化、缺失值及异常值的处理等，但是对于XGBoost模型而言，很多预处理都是不需要的，例如对于缺失值而言，XGBoost模型会自动处理，它会通过枚举所有缺失值在当前节点是进入左子树还是右子树来决定缺失值的处理方式。  

此外由于XGBoost是基于决策树模型，因此区别于线性回归等模型，像一些特征变换（例如离散化、归一化或者叫作标准化、取log、共线性问题处理等）都不太需要，这也是树模型的一个优点。如果有的读者还不太放心，可以自己尝试下做一下特征变换，例如数据归一化，会发现最终的结果都是一样的。这里给大家简单示范一下，通过如下代码对数据进行Z-score标准化或者叫作归一化。

In [24]:
from sklearn.preprocessing import StandardScaler
X_new = StandardScaler().fit_transform(X)

X_new  # 打印标准化后的数据

array([[-0.88269208, -1.04890243, -1.01409939, -0.60873764,  0.63591822],
       [-0.86319167,  0.09630122,  0.98609664, -1.55243002,  1.27956013],
       [-1.39227834, -1.46534013, -1.01409939, -0.83867808, -0.0077237 ],
       ...,
       [ 1.44337605,  0.61684833,  0.98609664,  1.01172301, -0.0077237 ],
       [ 0.63723633, -0.21602705,  0.98609664, -0.32732239, -0.0077237 ],
       [ 1.57656755,  0.61684833, -1.01409939,  1.30047599, -0.0077237 ]])

利用标准化后的数据进行建模，看看是否有差别：

In [25]:
# 3.划分测试集和训练集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=123)

# 4.建模
# 划分训练集和测试集完成后，就可以从Scikit-Learn库中引入XGBRegressor()模型进行模型训练了，代码如下：
from xgboost import XGBRegressor
model = XGBRegressor()  # 使用默认参数
model.fit(X_train, y_train)

# 因为XGBRegressor()是一个回归模型，所以通过查看R-squared来评判模型的拟合效果：
from sklearn.metrics import r2_score
r2 = r2_score(y_test, model.predict(X_test))
print(r2)

0.5716150813375576


此时再对这个X_new通过train_test_split()函数划分测试集和训练集，并进行模型的训练，最后通过r2_score()获得模型评分，会发现结果和没有归一化的数据的结果几乎一样，为0.571。这里也验证了树模型不需要进行特征的归一化或者说标准化，此外树模型对于共线性也不敏感。  

通过上面这个演示，也可以得出这么一个读者经常会问到的一个疑问：需不需要进行某种数据预处理？以后如果还有这样的疑问，那么不妨就做一下该数据预处理，如果发现最终结果没有区别，那就能够明白对于该模型不需要做相关数据预处理。  
当然绝大部分模型都无法自动完成的一步就是特征提取。很多自然语言处理的问题或者图象的问题，没有现成的特征，需要人工去提取这些特征。  

综上来说，XGBoost的确比线性模型要省去很多特征工程的步骤，但是特征工程依然是非常必要的，这一结论同样适用于下面即将讲到的LightGBM模型。